In [222]:
#import necessary packages
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import json
from sklearn.cluster import KMeans

# Define helper functions

In [8]:
def add_edge(adj_list, node, target):
    adj_list[int(node)].append(int(target)) 

In [9]:
def create_adj_list(data):
    #initialize adjacency lists
    min_node = data['FromNodeId'].min()
    max_node = data['FromNodeId'].max()

    adj_list_away = {}
    adj_list_toward = {}

    for node in range(min_node, max_node+1):
        adj_list_away[node] = []
        adj_list_toward[node] = []
        
    for idx, row in data.iterrows():
        add_edge(adj_list_away, row.FromNodeId, row.ToNodeId)
        add_edge(adj_list_toward, row.ToNodeId, row.FromNodeId)
        
    return adj_list_away, adj_list_toward

# Read in and clean the data

### City Reachability

In [10]:
city = pd.read_csv('City Reachability/reachability.txt', header = 5, delimiter = ' ')

#fix column names
city = city.rename(columns = {'#': 'FromNodeId', 'FromNodeId': 'ToNodeId', 'ToNodeId': 'Weight', 'Weight': 'redundant'})
city = city[city.columns[:-1]]

In [11]:
pd.concat([city['ToNodeId'], city['FromNodeId']]).unique().shape

(456,)

In [12]:
city

,FromNodeId,ToNodeId,Weight
0,27,0,-757
1,57,0,-84
2,70,0,-1290
3,74,0,-465
4,86,0,-700
...,...,...,...
71954,419,455,-154
71955,428,455,-341
71956,434,455,-403
71957,440,455,-680


In [26]:
#adjacency list away consists of edges going AWAY a certain node
#adjacency list toward consists of edges going TOWARD a certain node
city_adj_list_away, city_adj_list_toward = create_adj_list(city)

In [13]:
city_all_nodes = pd.concat([city['FromNodeId'], city['ToNodeId']]).unique()

### Stanford Web

In [16]:
stanford = pd.read_csv('Stanford Web/web-Stanford.txt', header = 3, delimiter = '\t')

In [17]:
stanford

,FromNodeId,ToNodeId
0,1,6548
1,1,15409
2,6548,57031
3,15409,13102
4,2,17794
...,...,...
2312492,281865,186750
2312493,281865,225872
2312494,281888,114388
2312495,281888,192969


In [9]:
#creating adj lists takes a while, avoid running if possible

#stanford_adj_list_away, stanford_adj_list_toward = create_adj_list(stanford)

In [27]:
#saved the adj lists for future use

with open('Stanford Web/adj_list_away.json', 'r') as fp:
    stanford_adj_list_away = json.load(fp)
    
with open('Stanford Web/adj_list_toward.json', 'r') as fp:
    stanford_adj_list_toward = json.load(fp)

In [18]:
stanford_all_nodes = pd.concat([stanford['FromNodeId'], stanford['ToNodeId']]).unique()

# Define counting functions (naive method)

In [169]:
def OLD_count_M1(adj_list_away, adj_list_toward):

    vertices = [] #store vertices
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in adj_list_away[vertex2]:
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 not in adj_list_toward[vertex2]) & (vertex2 not in adj_list_toward[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    #return triangles
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [170]:
def OLD_count_M2(adj_list_away, adj_list_toward):

    vertices = []
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in adj_list_away[vertex2]: #access all possible nodes (vertex 3) from vertex 2
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 not in adj_list_toward[vertex2]) & (vertex2 in adj_list_toward[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [171]:
def OLD_count_M3(adj_list_away, adj_list_toward):

    vertices = []
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in adj_list_away[vertex2]: #access all possible nodes (vertex 3) from vertex 2
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 in adj_list_toward[vertex2]) & (vertex2 in adj_list_toward[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [172]:
def OLD_count_M4(adj_list_away, adj_list_toward):

    vertices = []
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in adj_list_away[vertex2]: #access all possible nodes (vertex 3) from vertex 2
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 in adj_list_toward[vertex3])
                    & (vertex3 in adj_list_toward[vertex2]) & (vertex2 in adj_list_toward[vertex1])
                    & (vertex1 != vertex2) & (vertex1 != vertex3) & (vertex2 != vertex3)):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [173]:
def OLD_count_M5(adj_list_away, adj_list_toward):
    
    vertices = []
    
    for vertex1 in adj_list_away: #checks each parent node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            for vertex3 in adj_list_away[vertex1]:
                
                if ((vertex3 in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 not in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [174]:
def OLD_count_M6(adj_list_away, adj_list_toward):
    
    vertices = []
    
    for vertex1 in adj_list_away: #checks each parent node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in adj_list_away[vertex1]:
                
                if ((vertex3 in adj_list_away[vertex2]) & (vertex3 in adj_list_toward[vertex2])
                    & (vertex2 not in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [175]:
def OLD_count_M7(adj_list_away, adj_list_toward):
    
    vertices = []
    
    for vertex1 in adj_list_away: #checks each parent node
        for vertex2 in adj_list_toward[vertex1]: #checks first child node
            for vertex3 in adj_list_toward[vertex1]: #checks second child node
                
                if ((vertex2 not in adj_list_away[vertex1]) & (vertex3 not in adj_list_away[vertex1])
                   & (vertex2 in adj_list_away[vertex3]) & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [176]:
def OLD_count_M8(adj_list_away, adj_list_toward):

    vertices = []
    
    for vertex1 in adj_list_away: #checks each vertex1 node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            for vertex3 in adj_list_away[vertex1]: 

                if ((vertex3 not in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 not in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])
                    & (vertex2 != vertex3)):

                    vertices.append([vertex1, vertex2, vertex3])

    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle

    edge_dict = {}
    for tri in triangles:

        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else: #if edge does exist
                edge_dict[edge] += 1 #add to edge count

    return len(triangles), edge_dict

In [177]:
def OLD_count_M9(adj_list_away, adj_list_toward):
    
    vertices = []
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in adj_list_away[vertex2]: #access all possible nodes (vertex 3) from vertex 2
                
                if ((vertex1 not in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 not in adj_list_toward[vertex2]) & (vertex2 not in adj_list_toward[vertex1])
                    & (vertex1 != vertex3)):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [178]:
def OLD_count_M10(adj_list_away, adj_list_toward):
    
    vertices = []
    
    for vertex1 in adj_list_away: #checks first vertex
        for vertex2 in adj_list_toward[vertex1]: #checks second vertex
            for vertex3 in adj_list_toward[vertex1]:
                
                if ((vertex2 not in adj_list_away[vertex1]) & (vertex3 not in adj_list_away[vertex1])
                    & (vertex2 not in adj_list_away[vertex3]) & (vertex3 not in adj_list_away[vertex2])
                    & (vertex2 != vertex3)):

                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [179]:
def OLD_count_M11(adj_list_away, adj_list_toward):

    vertices = []
    
    for vertex1 in adj_list_away: #checks each vertex1 node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            for vertex3 in adj_list_away[vertex1]:

                if ((vertex3 not in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])
                    & (vertex2 != vertex3)):

                    vertices.append([vertex1, vertex2, vertex3])

    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle

    edge_dict = {}
    for tri in triangles:

        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else: #if edge does exist
                edge_dict[edge] += 1 #add to edge count

    return len(triangles), edge_dict

In [180]:
def OLD_count_M12(adj_list_away, adj_list_toward):
    
    vertices = []
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in adj_list_away[vertex2]:
                
                if ((vertex1 not in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 in adj_list_toward[vertex2]) & (vertex2 not in adj_list_toward[vertex1])
                    & (vertex1 != vertex3)):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [181]:
def OLD_count_M13(adj_list_away, adj_list_toward):

    vertices = []
    
    for vertex1 in adj_list_away: #checks each vertex1 node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            for vertex3 in adj_list_away[vertex1]:

                if ((vertex3 not in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 in adj_list_toward[vertex1]) & (vertex3 in adj_list_toward[vertex1])
                    & (vertex2 != vertex3)):

                    vertices.append([vertex1, vertex2, vertex3])

    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle

    edge_dict = {}
    for tri in triangles:

        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else: #if edge does exist
                edge_dict[edge] += 1 #add to edge count

    return len(triangles), edge_dict

# Define counting functions (sampling method)

In [101]:
def count_M1(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = [] #store vertices
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            for vertex3 in sampled_nodes:
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 not in adj_list_toward[vertex2]) & (vertex2 not in adj_list_toward[vertex1])
                    & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    #return triangles
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [114]:
def count_M2(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = [] #store vertices
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 not in adj_list_toward[vertex2]) & (vertex2 in adj_list_toward[vertex1])
                    & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [115]:
def count_M3(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = [] #store vertices
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 in adj_list_toward[vertex2]) & (vertex2 in adj_list_toward[vertex1])
                    & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [116]:
def count_M4(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex1 in adj_list_away[vertex3]) & (vertex1 in adj_list_toward[vertex3])
                    & (vertex3 in adj_list_toward[vertex2]) & (vertex2 in adj_list_toward[vertex1])
                    & (vertex1 != vertex2) & (vertex1 != vertex3) & (vertex2 != vertex3)
                    & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [117]:
def count_M5(all_nodes, adj_list_away, adj_list_toward, num_sample):
    
    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each parent node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex3 in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 not in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])
                    & (vertex3 in adj_list_away[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [118]:
def count_M6(all_nodes, adj_list_away, adj_list_toward, num_sample):
    
    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each parent node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex3 in adj_list_away[vertex2]) & (vertex3 in adj_list_toward[vertex2])
                    & (vertex2 not in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])
                    & (vertex3 in adj_list_away[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [119]:
def count_M7(all_nodes, adj_list_away, adj_list_toward, num_sample):
    
    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each parent node
        for vertex2 in adj_list_toward[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:  
                
                if ((vertex2 not in adj_list_away[vertex1]) & (vertex3 not in adj_list_away[vertex1])
                   & (vertex2 in adj_list_away[vertex3]) & (vertex3 in adj_list_away[vertex2])
                   & (vertex3 in adj_list_toward[vertex1])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [120]:
def count_M8(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each vertex1 node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes: 

                if ((vertex3 not in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 not in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])
                    & (vertex2 != vertex3) & (vertex3 in adj_list_away[vertex1])):

                    vertices.append([vertex1, vertex2, vertex3])

    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle

    edge_dict = {}
    for tri in triangles:

        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else: #if edge does exist
                edge_dict[edge] += 1 #add to edge count

    return len(triangles), edge_dict

In [121]:
def count_M9(all_nodes, adj_list_away, adj_list_toward, num_sample):
    
    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes: 
                
                if ((vertex1 not in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 not in adj_list_toward[vertex2]) & (vertex2 not in adj_list_toward[vertex1])
                    & (vertex1 != vertex3) & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [122]:
def count_M10(all_nodes, adj_list_away, adj_list_toward, num_sample):
    
    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks first vertex
        for vertex2 in adj_list_toward[vertex1]: #checks second vertex
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex2 not in adj_list_away[vertex1]) & (vertex3 not in adj_list_away[vertex1])
                    & (vertex2 not in adj_list_away[vertex3]) & (vertex3 not in adj_list_away[vertex2])
                    & (vertex2 != vertex3) & (vertex3 in adj_list_toward[vertex1])):

                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [123]:
def count_M11(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each vertex1 node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:

                if ((vertex3 not in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 in adj_list_toward[vertex1]) & (vertex3 not in adj_list_toward[vertex1])
                    & (vertex2 != vertex3) & (vertex3 in adj_list_away[vertex1])):

                    vertices.append([vertex1, vertex2, vertex3])

    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle

    edge_dict = {}
    for tri in triangles:

        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else: #if edge does exist
                edge_dict[edge] += 1 #add to edge count

    return len(triangles), edge_dict

In [124]:
def count_M12(all_nodes, adj_list_away, adj_list_toward, num_sample):
    
    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each starting vertex
        for vertex2 in adj_list_away[vertex1]: #access all possible nodes (vertex 2) from vertex 1
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:
                
                if ((vertex1 not in adj_list_away[vertex3]) & (vertex1 not in adj_list_toward[vertex3])
                    & (vertex3 in adj_list_toward[vertex2]) & (vertex2 not in adj_list_toward[vertex1])
                    & (vertex1 != vertex3) & (vertex3 in adj_list_away[vertex2])):
                    
                    vertices.append([vertex1, vertex2, vertex3])
                    
    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle
    
    edge_dict = {}
    for tri in triangles:
        
        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else:                            #if edge does exist
                edge_dict[edge] += 1 #add to edge count
    
    return len(triangles), edge_dict

In [125]:
def count_M13(all_nodes, adj_list_away, adj_list_toward, num_sample):

    vertices = []
    
    #randomly sample third vertices at uniform from all possible nodes
    sampled_nodes = np.random.choice(all_nodes, num_sample)
    
    for vertex1 in adj_list_away: #checks each vertex1 node
        for vertex2 in adj_list_away[vertex1]: #checks first child node
            
            #randomly sample a third vertex at uniform from all possible nodes
            for vertex3 in sampled_nodes:

                if ((vertex3 not in adj_list_away[vertex2]) & (vertex3 not in adj_list_toward[vertex2])
                    & (vertex2 in adj_list_toward[vertex1]) & (vertex3 in adj_list_toward[vertex1])
                    & (vertex2 != vertex3) & (vertex3 in adj_list_away[vertex1])):

                    vertices.append([vertex1, vertex2, vertex3])

    triangles = set(tuple(sorted(l)) for l in vertices) #get rid of permutations of the same triangle

    edge_dict = {}
    for tri in triangles:

        combos = list(itertools.combinations(tri, 2))

        for edge in combos:

            if edge not in edge_dict.keys(): #if edge doesn't exist yet
                edge_dict[edge] = 1
            else: #if edge does exist
                edge_dict[edge] += 1 #add to edge count

    return len(triangles), edge_dict

# Get motif counts (city reachability)

In [184]:
OLD_M1_count, OLD_edge_dict_M1 = OLD_count_M1(city_adj_list_away, city_adj_list_toward)

In [185]:
OLD_M1_count

232

In [126]:
M1_count, edge_dict_M1 = count_M1(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [127]:
M1_count

10

In [186]:
OLD_M2_count, OLD_edge_dict_M2 = OLD_count_M2(city_adj_list_away, city_adj_list_toward)

In [187]:
OLD_M2_count

13999

In [128]:
M2_count, edge_dict_M2 = count_M2(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [129]:
M2_count

104

In [188]:
OLD_M3_count, OLD_edge_dict_M3 = OLD_count_M3(city_adj_list_away, city_adj_list_toward)

In [189]:
OLD_M3_count

270284

In [130]:
M3_count, edge_dict_M3 = count_M3(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [131]:
M3_count

1931

In [190]:
OLD_M4_count, OLD_edge_dict_M4 = OLD_count_M4(city_adj_list_away, city_adj_list_toward)

In [191]:
OLD_M4_count

1497244

In [132]:
M4_count, edge_dict_M4 = count_M4(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [133]:
M4_count

65497

In [192]:
OLD_M5_count, OLD_edge_dict_M5 = OLD_count_M5(city_adj_list_away, city_adj_list_toward)

In [193]:
OLD_M5_count

1182

In [278]:
M5_count, edge_dict_M5 = count_M5(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [279]:
M5_count

3

In [194]:
OLD_M6_count, OLD_edge_dict_M6 = OLD_count_M6(city_adj_list_away, city_adj_list_toward)

In [195]:
OLD_M6_count

15111

In [136]:
M6_count, edge_dict_M6 = count_M6(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [137]:
M6_count

149

In [196]:
OLD_M7_count, OLD_edge_dict_M7 = OLD_count_M7(city_adj_list_away, city_adj_list_toward)

In [197]:
OLD_M7_count

16393

In [138]:
M7_count, edge_dict_M7 = count_M7(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [139]:
M7_count

189

In [198]:
OLD_M8_count, OLD_edge_dict_M8 = OLD_count_M8(city_adj_list_away, city_adj_list_toward)

In [199]:
OLD_M8_count

16013

In [140]:
M8_count, edge_dict_M8 = count_M8(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [141]:
M8_count

457

In [200]:
OLD_M9_count, OLD_edge_dict_M9 = OLD_count_M9(city_adj_list_away, city_adj_list_toward)

In [201]:
OLD_M9_count

25124

In [142]:
M9_count, edge_dict_M9 = count_M9(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [143]:
M9_count

199

In [202]:
OLD_M10_count, OLD_edge_dict_M10 = OLD_count_M10(city_adj_list_away, city_adj_list_toward)

In [203]:
OLD_M10_count

15280

In [144]:
M10_count, edge_dict_M10 = count_M10(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [145]:
M10_count

263

In [204]:
OLD_M11_count, OLD_edge_dict_M11 = OLD_count_M11(city_adj_list_away, city_adj_list_toward)

In [205]:
OLD_M11_count

349821

In [146]:
M11_count, edge_dict_M11 = count_M11(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [147]:
M11_count

5142

In [206]:
OLD_M12_count, OLD_edge_dict_M12 = OLD_count_M12(city_adj_list_away, city_adj_list_toward)

In [207]:
OLD_M12_count

338073

In [148]:
M12_count, edge_dict_M12 = count_M12(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [149]:
M12_count

3086

In [208]:
OLD_M13_count, OLD_edge_dict_M13 = OLD_count_M13(city_adj_list_away, city_adj_list_toward)

In [209]:
OLD_M13_count

2840251

In [150]:
M13_count, edge_dict_M13 = count_M13(city_all_nodes, city_adj_list_away, city_adj_list_toward, 5)

In [151]:
M13_count

73478

# Create the motif adjacency matrix

In [214]:
def create_motif_adjacency_matrix(edge_dict):
    
    #get all unique nodes
    first_half = [x[0] for x in edge_dict.keys()]
    second_half = [x[1] for x in edge_dict.keys()]
    combined = np.concatenate((first_half, second_half))
    
    unique_nodes = np.unique(combined)
    
    #define an empty dataframe to use as the motif adjacency matrix
    motif_adjacency_matrix = pd.DataFrame(index = unique_nodes, columns = unique_nodes)
    
    #populate motif adjacency matrix
    for key, value in edge_dict.items():
        motif_adjacency_matrix[key[0]][key[1]] = value
        motif_adjacency_matrix[key[1]][key[0]] = value

    motif_adjacency_matrix = motif_adjacency_matrix.fillna(0)
    
    return motif_adjacency_matrix.values

In [225]:
OLD_M10_matrix = create_motif_adjacency_matrix(OLD_edge_dict_M10)

# Get the single most optimal cluster

In [275]:
def single_cluster(adj_matrix):
    '''
    Input: motif adjacency matrix (W)
    Output:
    '''
    # Step 1: Form the Diagonal Matrix (D) from W
    # -----------------------------------------
    # obtain the sums of all the rows of W
    row_sums = np.sum(adj_matrix, axis = 1)
    
    # calculate D (to the -1/2 power) utilizing the row sums
    D = np.diag(row_sums**(-1/2))
    
    # Step 2: Calculate the normalized Laplacian matrix (L)
    # -----------------------------------------
    L = np.identity(len(row_sums)) - D@adj_matrix@D

    # Step 3: Calculate the eigenvector corrsponding to the second smallest eigenvalue of L
    # -----------------------------------------
    # calculate the eigenvalues and eigenvectors 
    eig = np.linalg.eig(L)
    eigenvalues, eigenvectors = eig[0], eig[1]

    # create sorted eigenvalue, eigenvector pairs
    eigenpairs = []
    for i in np.arange(len(eigenvalues)):
        eigenpairs.append((eigenvalues[i], eigenvectors[:,i]))
    
    # keep the eigenvector corresponding to the second smallest eigenvalue
    
    #return eigenpairs
    second_smallest_eigenvector = np.array(sorted(eigenpairs, key = lambda x: x[0])[1][1])
    
    # Step 4: Create a vector sigma whose sorted value index i corresponds to node i
    # -----------------------------------------
    sigma = D@second_smallest_eigenvector

    # Step 5: Linear sweep for motif conductance minimization
    # obtain the sorted indices of sigma where the ith index represents node i
    sorted_sigma = sorted(list(zip(sigma, np.arange(1, len(sigma)+1))))

    # obtain the proper orderings of the row and columns
    order = np.array([x[1] for x in sorted_sigma]) - 1

    # reorder the rows and columns of the adjacency matrix
    C = adj_matrix[order]
    C = C[:, order]

    # obtain the sum of each row
    row_sums = np.sum(C, 1)

    # calculate the volume of all clusters at each partition
    volumes = np.cumsum(row_sums)
    volumes_other = np.sum(np.sum(adj_matrix, 0)) * np.ones((len(order))) - volumes

    # calculate the conductances at each partition
    conductances = np.cumsum(row_sums - 2 * np.sum(np.tril(C), 1)) / np.minimum(volumes, volumes_other)
    conductances = np.nan_to_num(conductances, nan = 1)

    # find the index of the minimal motif conductance
    minimum_index = np.argmin(conductances)
    
    # partition the cluster which achieves the minimum conductance
    optimal_cluster = np.array([x[1] for x in sorted_sigma[:minimum_index + 1]])
    return optimal_cluster

In [276]:
OLD_M10_optimal = single_cluster(OLD_M10_matrix)

<ipython-input-275-a0ea5418f619>:57: RuntimeWarning: invalid value encountered in true_divide
  conductances = np.cumsum(row_sums - 2 * np.sum(np.tril(C), 1)) / np.minimum(volumes, volumes_other)


In [277]:
OLD_M10_optimal

array([319, 373, 336, 146,  18, 106, 121,  63,  11, 145, 198,  99, 141,
       249, 219,   5,  87,  71, 153, 113, 228, 412, 241,  51, 323,  77,
       415, 239, 205,  34,  82, 192, 385,  50, 438, 125, 424, 325,  22,
        95, 278, 215, 102, 439,   6, 226,  84, 254, 238,  55, 235,   3,
       223, 111, 123, 310, 377, 417,  85, 270, 158, 351, 220, 408,  76,
       201, 397, 311, 418, 164,  92, 288, 191, 407,  36, 208, 151,   7,
       331, 332, 165, 139, 202, 176, 117, 152, 276, 299, 360, 229, 272,
       428, 101,  60, 372, 135, 414, 359, 347,  78, 184, 324, 342, 368,
       337,  88, 395, 316, 289, 437, 433, 199, 195,  89, 390, 409,  74,
       133,  73, 329, 262, 108, 300, 116, 250,  14,  68, 284, 273,  28,
       394, 252,  86, 154, 148, 356, 222, 306, 149, 403, 296, 406,  54,
       376, 138, 304, 242, 341, 253,   1, 405, 328, 315, 298, 348,  97,
       308, 147, 258, 297, 231, 224, 178, 109, 429, 427, 277, 173, 114,
       357, 132, 322, 265, 268, 434, 419, 392, 321, 396, 378, 24

In [236]:
W = np.array([[0,3,1,1,1,0,0,0,0,0], 
              [3,0,1,1,1,1,1,0,0,0],
              [1,1,0,0,0,0,0,0,0,0],
              [1,1,0,0,0,0,0,0,0,0],
              [1,1,0,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,1,1,1,0],
              [0,1,0,0,0,1,0,0,0,0],
              [0,0,0,0,0,1,0,0,2,1],
              [0,0,0,0,0,1,0,2,0,1],
              [0,0,0,0,0,0,0,1,1,0]])

In [252]:
single_cluster(W)

<ipython-input-251-31a5165a1bd4>:57: RuntimeWarning: invalid value encountered in true_divide
  conductances = np.cumsum(row_sums - 2 * np.sum(np.tril(C), 1)) / np.minimum(volumes, volumes_other)


array([4, 5, 1, 3, 2])

# Testing Land